In [ ]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

# Constants for gesture recognition
THUMB_UP_THRESHOLD = 0.5
THUMB_DOWN_THRESHOLD = 0.6

while True:
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    h, w, c = img.shape
    results = hands.process(img)

    if results.multi_hand_landmarks:
        for hand_landmark in results.multi_hand_landmarks:
            lm_list = []
            for id, lm in enumerate(hand_landmark.landmark):
                lm_list.append(lm)
            finger_fold_status = []
            for tip in range(4, 21):
                x, y = int(lm_list[tip].x * w), int(lm_list[tip].y * h)
                cv2.circle(img, (x, y), 15, (255, 255, 255), cv2.FILLED)

                if tip == 4:
                    thumb_tip = lm_list[tip].x

                if lm_list[tip].x < lm_list[tip - 3].x:
                    cv2.circle(img, (x, y), 15, (0, 0, 0), cv2.FILLED)
                    finger_fold_status.append(True)
                else:
                    finger_fold_status.append(False)

            if all(finger_fold_status):
                # Like
                if thumb_tip > THUMB_UP_THRESHOLD:
                    cv2.putText(img, "Like", (250, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
                    print("LIKE")
                # Dislike
                elif thumb_tip < THUMB_DOWN_THRESHOLD:
                    cv2.putText(img, "Dislike", (250, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)
                    print("DISLIKE")

            mp_draw.draw_landmarks(img, hand_landmark,
                                   mp_hands.HAND_CONNECTIONS,
                                   mp_draw.DrawingSpec((0, 0, 0), 6, 3),
                                   mp_draw.DrawingSpec((255, 255, 255), 4, 2)
                                   )

    cv2.imshow("Hand Sign Detection", img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
